In [17]:
import findspark 
findspark.init() 

In [18]:
import pyspark 
from pyspark.sql import *
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.debug.maxToStringFields", 1000)
%load_ext sparksql_magic
%config SparkSql.limit=2000

The sparksql_magic extension is already loaded. To reload it, use:
  %reload_ext sparksql_magic


In [19]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV filcondae I/O (e.g. pd.read_csv)
import random as rd # generating random numbers
import matplotlib.pyplot as plt # basic plotting
import seaborn as sns # for prettier plots
import plotly.express as px
from pyspark.sql.types import StructType,StructField, StringType, IntegerType


In [20]:
path = "d:/lge/pycharm-projects/kaggle_store_sales/input"
train_master_all = spark.read.csv(f"{path}/train_master_all.csv", inferSchema = True, header = True);train_master_all.createOrReplaceTempView("train_master_all");spark.sql("CACHE TABLE train_master_all")

DataFrame[]

In [21]:
schema = StructType([
    StructField("work_dtm16"    , StringType()),
    StructField("scenario_id"   , StringType()),
    StructField("scenario_desc" , StringType()),
    StructField("feature_src"   , StringType()),
    StructField("feature_col"   , StringType()),
    StructField("feature_sdt8"  , StringType()),
    StructField("feature_edt8"  , StringType()),
    StructField("feature_desc"  , StringType()),
    StructField("predict_col"   , StringType()),
    StructField("predict_sdt8"  , StringType()),
    StructField("predict_edt8"  , StringType()),
    StructField("auto_ml_model" , StringType()),
    StructField("model_name"    , StringType()),
    StructField("model_cfg"     , StringType()),
    StructField("store_nbr"     , StringType()),
    StructField("family2"       , StringType()),
    StructField("test_y"        , StringType()),
    StructField("predict_y"     , StringType()),
    StructField("mae"           , StringType()),
    StructField("mse"          , StringType()),
    StructField("rmse"           , StringType()),
    StructField("msle"           , StringType()),
    StructField("rmsle"           , StringType()),
    StructField("r2"           , StringType()),
    StructField("score"         , StringType()),
    StructField("fit_tm_sec"    , StringType()),
    StructField("file_nm"       , StringType()),
    StructField("memo"          , StringType())
    ])

In [44]:
path = "d:/lge/pycharm-projects/kaggle_store_sales/output"
from os import listdir
from os.path import isfile, join
path = "d:/lge/pycharm-projects/kaggle_store_sales/output/"
files = [path+f for f in listdir(path) if isfile(join(path, f))]
results = spark.read.schema(schema).json(files);results.createOrReplaceTempView("results");spark.sql("CACHE TABLE results")
#results = spark.read.json(files);results.createOrReplaceTempView("results");spark.sql("CACHE TABLE results")


DataFrame[]

In [23]:
df = spark.sql('''
select scenario_id
     , count(*) cnt
     , count(case when score > 0.81 then 1 else null end) as s90       
     , count(case when score between 0.61 and 0.80 then 1 else null end) as s70       
     , count(case when score between 0.41 and 0.60 then 1 else null end) as s50       
     , count(case when score between 0.21 and 0.40 then 1 else null end) as s30      
     , count(case when score < 0.20 then 1 else null end) as s10
     , round(sum(fit_tm_sec)/60)  as fit_tm_min
from results       
where 1=1 
and scenario_id like'%t00%'
and scenario_id like'%t001%'
and scenario_id like'%x001%'
and scenario_id like'%d001%'
and scenario_id like'%m001%'
group by scenario_id
order by scenario_id
''')
pdf = df.toPandas()
pdf.head(20)


,scenario_id,cnt,s90,s70,s50,s30,s10,fit_tm_min
0,x001f004d001y001t001m001c001,1782,906,414,221,101,111,2.0
1,x001f005d001y001t001m001c001,1782,979,428,179,66,94,3.0
2,x001f006d001y001t001m001c001,1782,988,419,184,68,92,3.0
3,x001f007d001y001t001m001c001,1782,983,426,181,66,93,3.0
4,x001f008d001y001t001m001c001,1782,983,426,181,66,93,3.0
5,x001f009d001y001t001m001c001,1782,979,428,179,66,94,3.0
6,x001f010d001y001t001m001c001,1782,983,426,181,66,93,3.0


In [56]:
%%sparksql
--select store_nbr, family2, scenario_id,score,rmsle,predict_y,test_y,r2, mae,mse,rmse,msle
select avg(rmsle)
from results       
where 1=1 
and scenario_id like'x001f005d001y001t001m001c001'
and rmsle between = 0.11
--and rmsle = 0.11



avg(CAST(rmsle AS DOUBLE))
0.44101010101010085


In [52]:
%%sparksql
select rmsle, count(*) cnt
from results       
where 1=1 
and scenario_id like'x001f005d001y001t001m001c001'
group by rmsle
order by rmsle
limit 700

rmsle,cnt
0.0,127
0.1,1
0.11,2
0.12,1
0.13,2
0.14,9
0.15,5
0.16,9
0.17,17
0.18,33


In [47]:
%%sparksql
select store_nbr, family2, scenario_id,score,rmsle,predict_y,test_y,r2, mae,mse,rmse,msle
from results       
where 1=1 
and scenario_id like'x001f005d001y001t001m001c001'
--and store_nbr = 1
--and family2 = 1
--and scenario_id like'%f005%'
--and scenario_id like'%t002%'
--and rmsle is not null
--and family2 between 21 and 40
--and r2 between 0.1 and 0.3
--and rmsle between 0.3 and 0.5
order by cast(store_nbr as int), cast(family2 as int), rmsle 
--order by score desc, cast(store_nbr as int), cast(family2 as int), rmsle 
limit 70

store_nbr,family2,scenario_id,score,rmsle,predict_y,test_y,r2,mae,mse,rmse,msle
1,1,x001f005d001y001t001m001c001,0.73,0.52,"[5,4,4,4,4,4,5,5,4,4,4,4,3,5,4]","[5,4,3,8,5,6,7,4,7,9,1,6,1,1,4]",-0.06,2.0,6.27,2.5,0.27
1,2,x001f005d001y001t001m001c001,1.0,0.0,"[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]",1.0,0.0,0.0,0.0,0.0
1,3,x001f005d001y001t001m001c001,0.8,0.44,"[3,4,4,4,4,4,4,4,4,4,4,4,2,4,4]","[4,2,5,7,3,2,5,2,4,10,1,3,1,6,4]",0.08,1.73,5.07,2.25,0.2
1,4,x001f005d001y001t001m001c001,0.85,0.39,"[2616,2522,2328,2257,2151,2050,2586,2497,2408,2314,2207,2142,2041,2588,2511]","[2627,2645,2037,2479,2093,968,2086,2418,2311,2377,1006,1659,803,2201,1942]",-0.03,426.93,351652.4,593.0,0.15
1,5,x001f005d001y001t001m001c001,1.0,0.0,"[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]",1.0,0.0,0.0,0.0,0.0
1,6,x001f005d001y001t001m001c001,0.77,0.48,"[444,418,419,397,364,334,492,478,452,431,397,364,338,504,474]","[373,428,282,385,271,144,328,361,399,417,145,243,136,346,329]",-0.99,115.93,18488.73,135.97,0.23
1,7,x001f005d001y001t001m001c001,-0.12,1.06,"[21,20,19,18,17,16,23,22,21,20,18,17,16,24,22]","[11,7,10,24,12,2,18,11,12,17,4,3,1,4,21]",-1.61,9.93,124.07,11.14,1.12
1,8,x001f005d001y001t001m001c001,0.83,0.42,"[829,768,687,635,576,508,854,797,736,683,612,555,489,840,779]","[867,942,622,729,442,233,741,767,651,696,341,351,169,571,703]",0.39,144.07,30334.6,174.17,0.17
1,9,x001f005d001y001t001m001c001,0.78,0.47,"[897,859,822,784,746,707,935,897,859,821,782,744,706,934,895]","[668,941,529,767,624,312,717,569,789,736,343,526,266,699,602]",-1.12,230.93,70557.6,265.63,0.22
1,10,x001f005d001y001t001m001c001,0.88,0.35,"[149,140,129,176,114,99,157,148,141,133,157,111,103,160,149]","[133,150,106,170,104,60,138,130,123,139,64,99,47,154,116]",0.09,24.33,1109.4,33.31,0.12


In [25]:
# df = spark.sql('''
# with temp1 as (
#     select store_nbr, family2
#          , max(score) as score
#          , min(scenario_id) as scenario_id_min
#          , max(scenario_id) as scenario_id_max
#     from  (select * 
#            from results
#            where  length(predict_y) - length(replace(predict_y,',','')) + 1 > 10
#            )
#     where  1=1
#     and    scenario_id like'%t00%'
#     and    scenario_id like'%f005%'
#     and    scenario_id = replace(replace(replace(scenario_id,'t001','t002'),'d001','d003'),'d002','d004')
#     group by store_nbr, family2
#     order by cast(store_nbr as int), cast(family2 as int)
# ), temp2 as (
#     select a.store_nbr, a.family2, a.scenario_id, a.score, a.predict_y
#     from   results a,
#            temp1 b
#     where  a.store_nbr = b.store_nbr
#     and    a.family2   = b.family2
#     and    a.score     = b.score
#     order by a.store_nbr, a.family2
# ), temp3 as (
#     select a.store_nbr, a.family2, min(a.scenario_id) as scenario_id, a.score,replace(replace(replace(min(a.scenario_id),'t001','t002'),'d001','d003'),'d002','d004') as scenario_id2
#     from   temp2 a
#     where  1=1
#     --and    store_nbr = 20
#     --and    family2 = 1
#     group by a.store_nbr, a.family2, a.score
#     order by cast(a.store_nbr as int), cast(a.family2 as int), a.score
# ), temp4 as (
#     select a.store_nbr, a.family2, a.scenario_id, a.score, replace(replace(a.predict_y,'['),']') as predict_y
#     from   results a,
#            temp3 b
#     where  a.store_nbr = b.store_nbr
#     and    a.family2   = b.family2
#     and    a.scenario_id = b.scenario_id2
#     order by cast(a.store_nbr as int), cast(a.family2 as int)
# ), temp5 as (
#     select  '20170816' as date8, store_nbr, family2 ,split(predict_y,',')[0]  sales, predict_y from temp4 union all
#     select  '20170817' as date8, store_nbr, family2 ,split(predict_y,',')[1]  sales, predict_y from temp4 union all
#     select  '20170818' as date8, store_nbr, family2 ,split(predict_y,',')[2]  sales, predict_y from temp4 union all
#     select  '20170819' as date8, store_nbr, family2 ,split(predict_y,',')[3]  sales, predict_y from temp4 union all
#     select  '20170820' as date8, store_nbr, family2 ,split(predict_y,',')[4]  sales, predict_y from temp4 union all
#     select  '20170821' as date8, store_nbr, family2 ,split(predict_y,',')[5]  sales, predict_y from temp4 union all
#     select  '20170822' as date8, store_nbr, family2 ,split(predict_y,',')[6]  sales, predict_y from temp4 union all
#     select  '20170823' as date8, store_nbr, family2 ,split(predict_y,',')[7]  sales, predict_y from temp4 union all
#     select  '20170824' as date8, store_nbr, family2 ,split(predict_y,',')[8]  sales, predict_y from temp4 union all
#     select  '20170825' as date8, store_nbr, family2 ,split(predict_y,',')[9]  sales, predict_y from temp4 union all
#     select  '20170826' as date8, store_nbr, family2 ,split(predict_y,',')[10] sales, predict_y from temp4 union all
#     select  '20170827' as date8, store_nbr, family2 ,split(predict_y,',')[11] sales, predict_y from temp4 union all
#     select  '20170828' as date8, store_nbr, family2 ,split(predict_y,',')[12] sales, predict_y from temp4 union all
#     select  '20170829' as date8, store_nbr, family2 ,split(predict_y,',')[13] sales, predict_y from temp4 union all
#     select  '20170830' as date8, store_nbr, family2 ,split(predict_y,',')[14] sales, predict_y from temp4 union all
#     select  '20170831' as date8, store_nbr, family2 ,split(predict_y,',')[15] sales, predict_y from temp4
# ), temp6 as (
#     select  b.id, a.*, b.family
#     from    temp5 a, 
#             train_master_all b
#     where   a.store_nbr = b.store_nbr
#     and     a.family2   = b.family2
#     and     a.date8     = b.date8    
# ), temp7 as (
#     --select *
#     select id, sales
#     from   temp6 a
#     order by a.id, cast(a.store_nbr as int), cast(a.family2 as int), date8
# )
# select * from temp7
# ''')
# #df.show(5,False)
# pdf = df.toPandas()
# path = "d:/lge/pycharm-projects/kaggle_store_sales/output"
# pdf.to_csv(f'{path}/submission/submission_cat2007_20220426.csv', index=False)



In [26]:
path = "d:/lge/pycharm-projects/kaggle_store_sales/output"
submission_cat2007_20220426 = spark.read.csv(f"{path}/submission/submission_cat2007_20220426.csv", inferSchema = True, header = True);submission_cat2007_20220425.createOrReplaceTempView("submission_cat2007_20220426");spark.sql("CACHE TABLE submission_cat2007_20220426")

NameError: name 'submission_cat2007_20220425' is not defined

In [31]:
%%sparksql

--select store_nbr, family2, scenario_id, score, predict_y
select *
from   results
where  1=1
and scenario_id = 'x001f006d001y001t001m001c001'
--and    store_nbr = 1
--and    family2 = 1
and    scenario_id like'%f006%'
limit 20

work_dtm16,scenario_id,scenario_desc,feature_src,feature_col,feature_sdt8,feature_edt8,feature_desc,predict_col,predict_sdt8,predict_edt8,auto_ml_model,model_name,model_cfg,store_nbr,family2,test_y,predict_y,mae,mse,rmse,msle,rmsle,r2,score,fit_tm_sec,file_nm,memo
2022-04-30 09:40,x001f006d001y001t001m001c001,"[train_master_all][year4, season, day_of_week, onpromotion][20130101,20170730][sales][20170801,20170815][LinearRegression]",train_master_all,"year4, season, day_of_week, onpromotion",20130101,20170730,,sales,20170801,20170815,,LinearRegression,,1,1,"[5,4,3,8,5,6,7,4,7,9,1,6,1,1,4]","[4,4,4,4,4,3,5,4,4,4,4,4,3,5,4]",2.07,6.6,2.57,0.28,0.53,-0.11,0.72,0.121428,,
2022-04-30 09:40,x001f006d001y001t001m001c001,"[train_master_all][year4, season, day_of_week, onpromotion][20130101,20170730][sales][20170801,20170815][LinearRegression]",train_master_all,"year4, season, day_of_week, onpromotion",20130101,20170730,,sales,20170801,20170815,,LinearRegression,,1,2,"[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]",0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.097076,,
2022-04-30 09:40,x001f006d001y001t001m001c001,"[train_master_all][year4, season, day_of_week, onpromotion][20130101,20170730][sales][20170801,20170815][LinearRegression]",train_master_all,"year4, season, day_of_week, onpromotion",20130101,20170730,,sales,20170801,20170815,,LinearRegression,,1,3,"[4,2,5,7,3,2,5,2,4,10,1,3,1,6,4]","[2,4,4,4,4,3,4,4,4,4,4,4,2,4,4]",1.73,5.07,2.25,0.2,0.45,0.08,0.8,0.097,,
2022-04-30 09:40,x001f006d001y001t001m001c001,"[train_master_all][year4, season, day_of_week, onpromotion][20130101,20170730][sales][20170801,20170815][LinearRegression]",train_master_all,"year4, season, day_of_week, onpromotion",20130101,20170730,,sales,20170801,20170815,,LinearRegression,,1,4,"[2627,2645,2037,2479,2093,968,2086,2418,2311,2377,1006,1659,803,2201,1942]","[2593,2499,2299,2230,2123,2023,2561,2474,2386,2292,2186,2123,2023,2574,2499]",417.4,336941.8,580.47,0.15,0.38,0.01,0.85,0.098541,,
2022-04-30 09:40,x001f006d001y001t001m001c001,"[train_master_all][year4, season, day_of_week, onpromotion][20130101,20170730][sales][20170801,20170815][LinearRegression]",train_master_all,"year4, season, day_of_week, onpromotion",20130101,20170730,,sales,20170801,20170815,,LinearRegression,,1,5,"[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]",0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.114,,
2022-04-30 09:40,x001f006d001y001t001m001c001,"[train_master_all][year4, season, day_of_week, onpromotion][20130101,20170730][sales][20170801,20170815][LinearRegression]",train_master_all,"year4, season, day_of_week, onpromotion",20130101,20170730,,sales,20170801,20170815,,LinearRegression,,1,6,"[373,428,282,385,271,144,328,361,399,417,145,243,136,346,329]","[444,418,419,397,364,334,492,478,452,431,397,364,338,504,474]",115.93,18488.73,135.97,0.23,0.48,-0.99,0.77,0.099,,
2022-04-30 09:40,x001f006d001y001t001m001c001,"[train_master_all][year4, season, day_of_week, onpromotion][20130101,20170730][sales][20170801,20170815][LinearRegression]",train_master_all,"year4, season, day_of_week, onpromotion",20130101,20170730,,sales,20170801,20170815,,LinearRegression,,1,7,"[11,7,10,24,12,2,18,11,12,17,4,3,1,4,21]","[23,22,20,19,18,17,24,23,22,20,19,18,17,24,23]",10.8,143.6,11.98,1.21,1.1,-2.03,-0.21,0.101,,
2022-04-30 09:40,x001f006d001y001t001m001c001,"[train_master_all][year4, season, day_of_week, onpromotion][20130101,20170730][sales][20170801,20170815][LinearRegression]",train_master_all,"year4, season, day_of_week, onpromotion",20130101,20170730,,sales,20170801,20170815,,LinearRegression,,1,8,"[867,942,622,729,442,233,741,767,651,696,341,351,169,571,703]","[794,735,659,608,552,487,836,781,722,671,603,548,485,837,779]",141.6,29302.13,171.18,0.17,0.41,0.41,0.83,0.094999,,
2022-04-30 09:40,x001f006d001y001t001m001c001,"[train_master_all][year4, season, day_of_week, onpromotion][20130101,20170730][sales][20170801,20170815][LinearRegression]",train_master_all,"year4, season, day_of_week, onpromo

In [27]:
%%sparksql

select store_nbr, family2, scenario_id, score, predict_y
from   results
where  1=1
--and scenario_id = 'x003f005d003y001t002m003c001'
and    store_nbr = 1
and    family2 = 1
and    scenario_id like'%t002%'
limit 20

store_nbr,family2,scenario_id,score,predict_y
1,1,x003f005d003y001t002m001c001,-1.43,"[4,4,4,4,4,3,5,4,4,4,4,4,3,5,4,4,4]"
1,1,x003f005d004y001t002m001c001,-1.47,"[4,4,4,4,4,4,5,4,4,4,4,4,4,4,4,4,4]"
1,1,x003f005d003y001t002m003c001,-1.24,"[4,4,4,4,3,3,4,4,4,4,3,3,3,4,4,4,4]"
1,1,x003f005d004y001t002m003c001,-1.44,"[4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4]"
1,1,x001f005d003y001t002m001c001,-1.43,"[4,4,4,4,4,3,5,4,4,4,4,4,3,5,4,4,4]"
1,1,x001f005d004y001t002m001c001,-1.47,"[4,4,4,4,4,4,5,4,4,4,4,4,4,4,4,4,4]"
1,1,x001f005d003y001t002m003c001,-1.24,"[4,4,4,4,3,3,4,4,4,4,3,3,3,4,4,4,4]"
1,1,x001f005d004y001t002m003c001,-1.44,"[4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4]"
1,1,x003f005d003y001t002m002c001,-1.4,"[4,4,4,4,4,3,5,4,4,4,4,4,3,4,4,4,4]"
1,1,x003f005d004y001t002m002c001,-1.44,"[4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4]"


In [ ]:
%%sparksql
--select count(*) 
select *
from   submission_cat2007_20220426
where sales is null
order by id
limit 10


In [ ]:
%%sparksql
select * 
from   train_master_all
where  store_nbr = 20
and    family2 = 1
and date8 >= '20170816'
--and id in (3003066,3003074)
limit 5


In [ ]:
%%sparksql
select * 
from   results
where  store_nbr = 20
and family2 = 1

In [ ]:
path = "d:/lge/pycharm-projects/kaggle_store_sales/output"
pdf.to_csv(f'{path}/submission/submission_cat2007_20220425.csv', index=False)

In [ ]:
train_master_all
df = spark.sql('''
with temp1 as (
    select id,date,date10,date8,store_nbr,family2,family
    from   train_master_all
    where  date8 >= '20170816'
    order by id, store_nbr, family2
)
select *
from   temp1
order by cast(store_nbr as int)
''')
df.show(5,False)
# pdf = df.toPandas()
# pdf.to_csv(f'{path}/holidays_events_mart01.csv')



In [ ]:
spark.sql('''
select count(case when score < 0.20 then 1 else null end) as s10
     , count(case when score between 0.21 and 0.40 then 1 else null end) as s30      
     , count(case when score between 0.41 and 0.60 then 1 else null end) as s50       
     , count(case when score between 0.61 and 0.80 then 1 else null end) as s70       
     , count(case when score > 0.81 then 1 else null end) as s90       
from (
    select max(a.scenario_id) as scenario_id, a.store_nbr, a.family2, a.score 
    from   results a,
          ( select store_nbr, family2, max(score) score
            from results 
            group by store_nbr, family2
          ) b
    where a.store_nbr = b.store_nbr
    and   a.family2 = b.family2
    and   a.score = b.score
    group by a.store_nbr, a.family2, a.score
    order by a.store_nbr, a.family2, a.score
    )
''').show(2000,False)

In [ ]:
%%sparksql
select scenario_id, store_nbr, family2, score 
--select *
from results 
where 1=1
and scenario_id like'%m001%'
--and store_nbr = '1' 
--and family2 = '2'
order by substr(scenario_id,13,4), substr(scenario_id,1,4), substr(scenario_id,5,4)
limit 2


In [ ]:
%%sparksql
select *
from results 
where 1=1
--and scenario_id like'%m001%'
and scenario_id = 'x002f005d003y001t002m001c001'
and store_nbr = '1' 
and family2 = '1'
limit 20

In [ ]:
spark.sql('''
select scenario_id, count(*)
from (
    select max(a.scenario_id) as scenario_id, a.store_nbr, a.family2, a.score 
    from   results a,
          ( select store_nbr, family2, max(score) score
            from results 
            group by store_nbr, family2
          ) b
    where a.store_nbr = b.store_nbr
    and   a.family2 = b.family2
    and   a.score = b.score
    group by a.store_nbr, a.family2, a.score
    order by a.store_nbr, a.family2, a.score
     )
where score > 0.81     
group by scenario_id      
order by scenario_id      
''').show(2,False)

In [ ]:
%%sparksql

select scenario_id
     , count(*) cnt
     , count(case when score < 0.20 then 1 else null end) as s10
     , count(case when score between 0.21 and 0.40 then 1 else null end) as s30      
     , count(case when score between 0.41 and 0.60 then 1 else null end) as s50       
     , count(case when score between 0.61 and 0.80 then 1 else null end) as s70       
     , count(case when score > 0.81 then 1 else null end) as s90       
     , max(scenario_desc) as scenario_desc
     , max(feature_col ) as feature_col
     , max(feature_sdt8) as feature_sdt8
     , max(feature_edt8) as feature_edt8
     , max(predict_col ) as predict_col
     , max(predict_sdt8) as predict_sdt8
     , max(predict_edt8) as predict_edt8
     , round(sum(fit_tm_sec)/60)  as fit_tm_min
from results                                                                                                                                                                     
group by scenario_id
order by scenario_id
limit 2

In [ ]:
spark.sql('''
select auto_ml_model, count(*)
from results
where scenario_id like '%m004'
group by auto_ml_model
order by auto_ml_model
''').show(2,False)